# Modeling

### Getting encoded data

Since we have to do feature selection again we have to do one hot encoding again

In [1]:

import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import tflearn

C:\Users\dokee\anaconda3\envs\TensorEnv\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\dokee\anaconda3\envs\TensorEnv\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\dokee\anaconda3\envs\TensorEnv\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\dokee\anaconda3\envs\TensorEnv\lib\site-packages\ten

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


C:\Users\dokee\anaconda3\envs\TensorEnv\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\dokee\anaconda3\envs\TensorEnv\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\dokee\anaconda3\envs\TensorEnv\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\dokee\anaconda3\envs\TensorEnv\

In [2]:
cleaned_df = pd.read_csv("cleaned_df.csv") 

In [3]:
cleaned_df = cleaned_df.drop(["COVID_HR1","COVID_HR2","IDEOLOGY4"],axis=1)
missing = ["HR11","IDEOLOGY7"]
for i in missing:
    mode = cleaned_df[i].mode()
    print(mode[0])
    fixed = []
    for value in cleaned_df[i]:
        if value == 'NA' or value != value:
            fixed.append(mode[0])
        else:
            fixed.append(value)
    cleaned_df[i] = fixed

B
C


In [4]:
for (columnName, columnData) in cleaned_df.iteritems(): 
    missingCount = (columnData == 'NA').sum() 
    missingCount += columnData.isnull().sum() 
    percentage = (missingCount/len(cleaned_df))*100 
    #print(percentage)
    if missingCount > 0:        
        print("Missing Values in column : ", columnName, ", missing rows: ", 
            missingCount, ", missing percentage: ",  percentage, '%') 
        if percentage > 50.0:
            too_many_missing_value_columns.append(columnName) 

### Feature selection
Feature we are going to use here is the ordinal IDEOLOGY5. IDEOLOGY5 is "If you had to describe your ideology along a Left-Right political spectrum, which of the following would come closest to how you would describe yourself?"

In [5]:
cleaned_df["IDEOLOGY5"].unique()

array([ 2,  0, -2, -1,  1], dtype=int64)

In [5]:
tmp_df = cleaned_df.copy()
nominal_val = cleaned_df.select_dtypes(include=['object']).columns
for index, values in tmp_df.iteritems():
    if index in nominal_val:
        unique = pd.unique(values)
        dic = []
        for i in values:
            dic.append(np.where(unique == i)[0][0])
        tmp_df[index] = dic

In [6]:
tmp_df.columns

Index(['Unnamed: 0', 'ARM', 'HR1_10SCALE', 'HR2_1', 'HR2_2', 'HR2_3', 'HR2_4',
       'HR2_5', 'HR3', 'HR4',
       ...
       'RUMORS1_USARMY_BROUGHT', 'RUMORS1_BAT_TO_HUMANS',
       'RUMORS1_BAT_ANIMAL_HUMANS', 'RUMORS1_WET_MARKET', 'RUMORS1_OTHER',
       'DEMOGRAPHIC1_EMPLOYED', 'DEMOGRAPHIC1_UNEMPLOYED',
       'DEMOGRAPHIC1_RETIRED', 'DEMOGRAPHIC1_STUDENTS', 'DEMOGRAPHIC1_OTHER'],
      dtype='object', length=216)

In [12]:
from sklearn.model_selection import train_test_split
X = tmp_df.drop(["IDEOLOGY5","HR6"],axis=1) #obamacare
y = tmp_df["HR6"]
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size = .75, test_size = .25, random_state = 72)

### Feature selection with chi squared

In [13]:
from sklearn.feature_selection import SelectKBest, chi2
fs = SelectKBest(score_func=chi2, k='all')
fs.fit(x_train,y_train)
x_train_fs = fs.transform(x_train)
x_test_fs = fs.transform(x_test)

In [17]:
selected_features = []
for i in range(len(fs.scores_)):
    #print('Feature %s: %f' % (tmp_df.columns[i], fs.scores_[i]))
    if fs.scores_[i] > 100:
        print('Feature %s: %f' % (X.columns[i], fs.scores_[i]))
        selected_features.append(X.columns[i])
selected_features.remove("Unnamed: 0")
selected_features

Feature Unnamed: 0: 1118.174156
Feature HR1_10SCALE: 116.613315
Feature HR2_1: 186.493677
Feature HR2_2: 229.000817
Feature HR4: 226.407734
Feature HR5: 173.098587
Feature HR7_1: 120.794659
Feature HR7_2: 109.100488
Feature HR7_3: 137.264634
Feature HR7_4: 115.881932
Feature HR8: 105.482972
Feature STIMULUS3: 126.266666
Feature STIMULUS4: 125.173555
Feature PERSONAL3: 159.764489
Feature MEDIA10: 175.402413
Feature MEDIA12: 175.526229
Feature CHILDCARE3: 101.606536
Feature CULTURE2_1: 118.960475
Feature DEMOGRAPHIC4: 118.540447
Feature DEMOGRAPHIC5: 6782.521443
Feature age: 454.866102
Feature STIMULUS1_Nothing changed: 128.030669


['HR1_10SCALE',
 'HR2_1',
 'HR2_2',
 'HR4',
 'HR5',
 'HR7_1',
 'HR7_2',
 'HR7_3',
 'HR7_4',
 'HR8',
 'STIMULUS3',
 'STIMULUS4',
 'PERSONAL3',
 'MEDIA10',
 'MEDIA12',
 'CHILDCARE3',
 'CULTURE2_1',
 'DEMOGRAPHIC4',
 'DEMOGRAPHIC5',
 'age',
 'STIMULUS1_Nothing changed']

Selected Features are 'HR6', 'MEDIA12', 'IDEOLOGY6', 'DEMOGRAPHIC4', 'DEMOGRAPHIC5' DEMOGRAPHIC 4 is nominal so it will have to be encoded.

### One Hot

In [24]:
#selected_df = cleaned_df[selected_features].copy()
selected_df["HR6"] = cleaned_df["HR6"].copy()
selected_df

,HR1_10SCALE,HR2_1,HR2_2,HR4,HR5,HR7_1,HR7_2,HR7_3,HR7_4,HR8,...,DEMOGRAPHIC5_31323,DEMOGRAPHIC5_90005,DEMOGRAPHIC5_18051,DEMOGRAPHIC5_25271,DEMOGRAPHIC5_43701,DEMOGRAPHIC5_34475,DEMOGRAPHIC5_6152,DEMOGRAPHIC5_21740,DEMOGRAPHIC5_40353,HR6
0,4,3,1,2,3,2,1,2,2,2,...,0,0,0,0,0,0,0,0,0,3
1,4,3,3,2,3,2,2,1,2,2,...,0,0,0,0,0,0,0,0,0,2
2,10,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,5,3,3,1,2,2,2,1,3,1,...,0,0,0,0,0,0,0,0,0,2
4,5,3,0,4,1,4,0,1,3,0,...,0,0,0,0,0,0,0,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1206,5,1,2,4,4,4,4,4,4,3,...,0,0,0,0,1,0,0,0,0,3
1207,1,5,5,4,4,4,4,4,4,3,...,0,0,0,0,0,1,0,0,0,4
1208,4,4,4,3,3,1,1,0,2,2,...,0,0,0,0,0,0,1,0,0,2
1209,1,5,5,4,4,4,1,1,3,3,...,0,0,0,0,0,0,0,1,0,4


In [19]:
nominal_df = selected_df[selected_df.select_dtypes(include=['object']).columns]
nominal_df

,CHILDCARE3,DEMOGRAPHIC4,DEMOGRAPHIC5
0,No,IN,46825
1,No,CO,81650
2,No,GA,31027
3,No,IL,60609
4,No,MS,39466
...,...,...,...
1206,No,OH,43701
1207,No,DE,34475
1208,No,IL,6152
1209,No,MD,21740


In [20]:
nn_df = pd.DataFrame()
for i in nominal_df.columns:
    unique = pd.unique(selected_df[i].copy())
    count = 0
    for j in unique:
        name = i + "_" + str(j)
        tmp = []
        for k in nominal_df[i]:
            if k == j:
                tmp.append(1)
            else:
                tmp.append(0)
        count +=1
        nn_df[name] = tmp
nn_df

,CHILDCARE3_No,CHILDCARE3_Yes,DEMOGRAPHIC4_IN,DEMOGRAPHIC4_CO,DEMOGRAPHIC4_GA,DEMOGRAPHIC4_IL,DEMOGRAPHIC4_MS,DEMOGRAPHIC4_TX,DEMOGRAPHIC4_WA,DEMOGRAPHIC4_TN,...,DEMOGRAPHIC5_12225,DEMOGRAPHIC5_31323,DEMOGRAPHIC5_90005,DEMOGRAPHIC5_18051,DEMOGRAPHIC5_25271,DEMOGRAPHIC5_43701,DEMOGRAPHIC5_34475,DEMOGRAPHIC5_6152,DEMOGRAPHIC5_21740,DEMOGRAPHIC5_40353
0,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1206,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1207,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1208,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1209,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [21]:
selected_df = selected_df.drop(nominal_df.columns,axis=1)
for index, value in nn_df.iteritems():
    selected_df[index] = value

In [23]:
selected_df["HR6"]

KeyError: 'HR6'

### Now we have our selected features for our data. 

In [25]:
selected_df.to_csv("HR6_encoded.csv")
selected_df

,HR1_10SCALE,HR2_1,HR2_2,HR4,HR5,HR7_1,HR7_2,HR7_3,HR7_4,HR8,...,DEMOGRAPHIC5_31323,DEMOGRAPHIC5_90005,DEMOGRAPHIC5_18051,DEMOGRAPHIC5_25271,DEMOGRAPHIC5_43701,DEMOGRAPHIC5_34475,DEMOGRAPHIC5_6152,DEMOGRAPHIC5_21740,DEMOGRAPHIC5_40353,HR6
0,4,3,1,2,3,2,1,2,2,2,...,0,0,0,0,0,0,0,0,0,3
1,4,3,3,2,3,2,2,1,2,2,...,0,0,0,0,0,0,0,0,0,2
2,10,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,5,3,3,1,2,2,2,1,3,1,...,0,0,0,0,0,0,0,0,0,2
4,5,3,0,4,1,4,0,1,3,0,...,0,0,0,0,0,0,0,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1206,5,1,2,4,4,4,4,4,4,3,...,0,0,0,0,1,0,0,0,0,3
1207,1,5,5,4,4,4,4,4,4,3,...,0,0,0,0,0,1,0,0,0,4
1208,4,4,4,3,3,1,1,0,2,2,...,0,0,0,0,0,0,1,0,0,2
1209,1,5,5,4,4,4,1,1,3,3,...,0,0,0,0,0,0,0,1,0,4
